<title>Applied Science Capstone Project</title>

<h1>Importing the necessary libraries.</h1>

In [1]:
#Scraping Data from Wiki into JSON
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

<h1>Create settings for Foursqure. Set centre of the map to Dublin city centre.</h1>

In [4]:
CLIENT_ID = 'KLFE0G1UUPTOQYV40F31ITB1IMS0SERM4YOKNU0X4ADX42ET' # your Foursquare ID
CLIENT_SECRET = '0J0UWRK13VP0AHYHX3BPEDXQYPZUJMPIKQVS4M1DWUFLCIUG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# type your answer here
radius = 500
LIMIT = 100

dublin_latitude = 53.3498
dublin_longitude = -6.2603

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dublin_latitude, dublin_longitude, VERSION, radius, LIMIT)
url


Your credentails:
CLIENT_ID: KLFE0G1UUPTOQYV40F31ITB1IMS0SERM4YOKNU0X4ADX42ET
CLIENT_SECRET:0J0UWRK13VP0AHYHX3BPEDXQYPZUJMPIKQVS4M1DWUFLCIUG


'https://api.foursquare.com/v2/venues/search?client_id=KLFE0G1UUPTOQYV40F31ITB1IMS0SERM4YOKNU0X4ADX42ET&client_secret=0J0UWRK13VP0AHYHX3BPEDXQYPZUJMPIKQVS4M1DWUFLCIUG&ll=53.3498,-6.2603&v=20180605&radius=500&limit=100'

<h1> Test the Foursqare setting</h1>

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da8e508a30619002cf5e0ce'},
 'response': {'venues': [{'id': '4afd215ff964a520052722e3',
    'name': 'The Spire of Dublin / An Túr Solais (The Spire of Dublin)',
    'location': {'address': "O'Connell St",
     'crossStreet': 'Monument of Light',
     'lat': 53.34980521905305,
     'lng': -6.260259747505188,
     'labeledLatLngs': [{'label': 'display',
       'lat': 53.34980521905305,
       'lng': -6.260259747505188}],
     'distance': 2,
     'postalCode': '1',
     'cc': 'IE',
     'city': 'Dublin',
     'state': 'Dublin City',
     'country': 'Ireland',
     'formattedAddress': ["O'Connell St (Monument of Light)",
      'Dublin',
      'Dublin City',
      'Ireland']},
    'categories': [{'id': '4bf58dd8d48988d12d941735',
      'name': 'Monument / Landmark',
      'pluralName': 'Monuments / Landmarks',
      'shortName': 'Landmark',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_monument_',
       'suffix': '

<h1>Extract the categories of the Venues</h1>

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['venues']

nearby_venues = json_normalize(venues) # flatten JSON
#nearby_venues
# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']

nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Spire of Dublin / An Túr Solais (The Spire...,Monument / Landmark,53.349805,-6.260260
1,Kingdom Of Sweets,Candy Store,53.349578,-6.261536
2,Butlers Chocolate Café,Café,53.349757,-6.260579
3,Griffin's Londis,Convenience Store,53.349811,-6.260358
4,MoneyGram,Bank,53.350138,-6.260067


In [9]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<h1>Create a function to extract information of all of the townlands</h1>

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h1>Manually create the Dataframe that contains the locations of the townlands</h1>

In [11]:
dublin_15_data = pd.DataFrame(np.array([['Castleknock', 53.374, -6.359], 
                                     ['Clonsilla', 53.383333, -6.416667], 
                                     ['Cloghran', 53.4119, -6.3572], 
                                     ['Blanchardstown', 53.3842, -6.3760], 
                                     ['Cabra', 53.365714, -6.290789], 
                                     ['Corduff', 53.398057, -6.371681], 
                                     ['Dunsink', 53.387338, -6.33849], 
                                     ['Huntstown and Littlepace', 53.387, -6.38], 
                                     ['Mulhuddart', 53.4028,  -6.398157], 
                                     ['Tyrrelstown', 53.419922, -6.383103], 
                                     ['Finglas', 53.390175, -6.299629]]),
                           columns=['District', 'Latitude', 'Longitude'])

#Call the created function to gather information on the townlands

dublin_15_venues = getNearbyVenues(names=dublin_15_data['District'],
                                   latitudes=dublin_15_data['Latitude'],
                                   longitudes=dublin_15_data['Longitude']
                                  )

Castleknock
Clonsilla
Cloghran
Blanchardstown
Cabra
Corduff
Dunsink
Huntstown and Littlepace
Mulhuddart
Tyrrelstown
Finglas


In [12]:
print(dublin_15_venues.shape)
dublin_15_venues

(71, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Castleknock,53.374,-6.359,The Butler's Pantry,53.372938,-6.361444,Bakery
1,Castleknock,53.374,-6.359,Silver Spoon,53.375007,-6.363494,Café
2,Castleknock,53.374,-6.359,Myo's Pub,53.372869,-6.362241,Pub
3,Castleknock,53.374,-6.359,Lidl,53.373376,-6.362627,Supermarket
4,Castleknock,53.374,-6.359,The Mandarin Castle,53.372790,-6.361430,Chinese Restaurant
5,Castleknock,53.374,-6.359,Okra Green,53.372763,-6.361539,Asian Restaurant
6,Castleknock,53.374,-6.359,Castello Bruno,53.372359,-6.360571,Italian Restaurant
7,Castleknock,53.374,-6.359,De Brúns,53.372913,-6.360671,Gastropub
8,Castleknock,53.374,-6.359,Millennium 2,53.372955,-6.360639,Chinese Restaurant
9,Castleknock,53.374,-6.359,Educogym,53.372891,-6.360680,Gym


In [13]:
dublin_15_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Blanchardstown,14,14,14,14,14,14
Cabra,5,5,5,5,5,5
Castleknock,11,11,11,11,11,11
Cloghran,1,1,1,1,1,1
Clonsilla,5,5,5,5,5,5
Corduff,4,4,4,4,4,4
Finglas,6,6,6,6,6,6
Huntstown and Littlepace,9,9,9,9,9,9
Mulhuddart,7,7,7,7,7,7


<h1>Create the map using Folium library and make all of the venues</h1>

In [17]:
# create map
#filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']

dublin_latitude = 53.3498
dublin_longitude = -6.2603
map_dublin = folium.Map(location=[dublin_latitude, dublin_longitude], zoom_start=12)

# add markers to map
for lat, lng, name, categories in zip(dublin_15_venues['Venue Latitude'], dublin_15_venues['Venue Longitude'], dublin_15_venues['Venue'], dublin_15_venues['Venue Category']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)  
    
map_dublin

<h1>Remove the columns that contains strings. Make the dataframe ready for K-Nearest Neighbour.</h1>

In [18]:
d15_location = dublin_15_venues.reset_index(drop = True) 

d15_location = d15_location.drop(['Neighborhood', 'Venue', 'Venue Category'], 1)
d15_location.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,53.374,-6.359,53.372938,-6.361444
1,53.374,-6.359,53.375007,-6.363494
2,53.374,-6.359,53.372869,-6.362241
3,53.374,-6.359,53.373376,-6.362627
4,53.374,-6.359,53.372790,-6.361430


<h1>Train the clustering model with the dataframe</h1>

In [19]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(d15_location)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
k_means_cluster_centers = kmeans.cluster_centers_[:,2:4]



<h1>Insert the clustering lables (created by the model) back to the dataframe</h1>

In [20]:
# add clustering labels
dublin_15_venues.insert(0, 'Cluster Labels', kmeans.labels_)

In [21]:
#dublin_15_venues
print(k_means_cluster_centers.shape)
k_means_cluster_centers

(5, 2)


array([[53.37307685, -6.36152385],
       [53.38705661, -6.37594968],
       [53.37832271, -6.29496985],
       [53.41261915, -6.38626635],
       [53.3829976 , -6.42114601]])

<h1>Create the map with the clustering coloured and marked the centre with map marker</h1>

In [22]:
# create map
d15_latitude = 53.3830
d15_longitude = -6.4165
map_clusters = folium.Map(location=[d15_latitude, d15_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dublin_15_venues['Venue Latitude'], dublin_15_venues['Venue Longitude'], dublin_15_venues['Venue'], dublin_15_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)

tooltip = 'Suggested EV Charging Station'
for location in k_means_cluster_centers:
    folium.Marker(location, popup='<i>Suggested EV Charging Station</i>').add_to(map_clusters)

map_clusters